# plot time series data from SVD analyses

This page assumes that an SVD analysis has been conducted of the project in this folder, and you now wish to simply plot the colour-coded time series from the `images` datacube, with `eigenvalues` providing the colour code.

In [1]:
%matplotlib tk
import pystac_client, pickle, pyperclip
import odc.stac
import dask.distributed
import dask.array
import numpy as np
import geopandas as gpd
from IPython.display import JSON
import matplotlib.pyplot as plt
import rasterio as rio
import xarray as xr

List the pickle files to see where `images` and `dates` will come from:

In [2]:
%ls -lh *.pkl

 Volume in drive C is Windows-SSD
 Volume Serial Number is 8680-F1B8

 Directory of c:\Users\mathe\DEPWS\katherine\basemap


 Directory of c:\Users\mathe\DEPWS\katherine\basemap

16/10/2023  02:16 PM     4,338,261,481 ndvi-l8-30m.pkl
16/10/2023  02:16 PM             4,082 ndvi-l8-30m-dates.pkl
15/10/2023  07:56 PM             1,350 ndvi-mFrac-20231015.pkl
17/01/2024  12:46 PM               227 ndvi-mFrac-20231015-30m.pkl
16/10/2023  02:31 PM     4,338,261,481 ndvi-s2-10m-eigs.pkl
17/01/2024  12:55 PM       258,230,005 ndvi-s2-30m.pkl
17/01/2024  12:55 PM             2,812 ndvi-s2-30m-dates.pkl
17/01/2024  01:05 PM       258,230,005 ndvi-s2-30m-eigs.pkl
               8 File(s)  9,192,991,443 bytes
               0 Dir(s)  682,628,554,752 bytes free


The image transformation and the eigenvector values come from a selected `svdComposite*.tif` layer in the project.  Refer to your `qgz` project to know which one to choose, and refer to it below:

In [3]:
#load all the visuals if needed
#with open("ndvi-s2-10m-eigs.pkl", 'rb') as inf:
#    eigs = pickle.load(inf)
with open("ndvi-s2-30m-dates.pkl", 'rb') as inf:
    dates = pickle.load(inf)
with open("ndvi-s2-30m.pkl", 'rb') as inf:
    images = pickle.load(inf)
with rio.open("eigenlayers-s2-30.tif") as src:
    sTrans = src.transform
    eigs = src.read()

In [4]:
# get a color map of the three eigenvalues scaled to their eigenlayers
# assign which eigenlayers go to each band
cmp = {'red': 0, 'green': 1, 'blue': 2}
flip = (1,-1, 1)
eigr = lambda e: eigs[cmp[e]]
    
# stretch the range of the color band to +- 2 stddev
stdRange = lambda e: {'min': eigr(e).mean() - eigr(e).std() * 2 * flip[cmp[e]],
                      'max': eigr(e).mean() + eigr(e).std() * 2 * flip[cmp[e]]}
# assign these ranges to a hash
ranges = {i: stdRange(i) for i in cmp.keys()}
    
sclr = lambda  cl: (eigr(cl)[mr, mc] - ranges[cl]['min']) / \
                   (ranges[cl]['max'] - ranges[cl]['min'])


def scmpfinder (mr, mc):
    # how to compute a particular eigenvalue as a fraction of its range

    # how to compute a particular eigenvalue as a fraction of its range
    #sclr = lambda  cl: (eigs[cmp[cl], mr, mc] - ranges[cl]['min']) / \
    #        (ranges[cl]['max'] - ranges[cl]['min'])
          
    # create RGB colour list of the three ranges
    scmp = [sclr(i) for i in cmp.keys()]
    #scmp =  [sclr('red'), sclr('green'), sclr('blue')]
    # and chop the bits that exceed [0:1]
    for i, v in enumerate(scmp):
        if v < 0.0: scmp[i] = 0.0
        if v > 1.0: scmp[i] = 1.0
        
    return scmp


In [8]:
# get the dates from the series
#dates = [i.properties.date for i in gScenes]
# set the plot limit to a fixed range
plt.ylim(-.3,1)
# get the matrix diminsions from the saved tif file
#with rio.open("max_ndvi.tif") as src:
#sTrans = src_meta['transform']
sTrans = src.transform

# get the coordinates I wish to plot (currently in UTM53N)
#coords = 280929.5,8361645.4
# this gets the coordinates from the clipboard from QGIS.  It comes
# as a string of two numbers separated by a comma, this turns it into 
# a pair of floating point numbers
coords = [float(s) for s in pyperclip.paste().split(',')]
print(coords)
# translate these into row, col based on the raster tranform matrix
# rowcol = rio.transform.rowcol(sTrans,*coords)
rowcol = lambda r, c: rio.transform.rowcol(sTrans, r, c)
print(rowcol(*coords))

mr, mc = rowcol(*coords)
# get the corresponding time series from the images matrix and plot it
#mages[:, mr, mc]
# get the colormap according to its eigenlayers, and plot
scmp = scmpfinder(mr, mc)
plt.plot(dates, np.ma.getdata(images[:, mr, mc]), color=scmp)

plt.axhline(0, linestyle=':')


[197642.0, 8386894.0]
(957, 2062)


In [6]:
plt.clf()

In [7]:
pyperclip.paste()

'226381,8369924'